In [ ]:
# extract Breast profile 

import numpy as np
import cv2
from skimage.measure import label, regionprops


class Preprocess:
    def __init__(self, rawim, im, breast_mask, lesion_mask):
        self.raw = rawim
        self.image = im
        self.mask = breast_mask
        self.lesion_mask = lesion_mask
   
    def extract_breast_profile(image,lesion_mask, if_crop):
        
        breast_mask = np.zeros(np.shape(image))
        breast_mask[image>0]=1
        
        labelim = label(breast_mask)
        props =  regionprops(labelim)
#        find the largest object as the breast
        area = 0
        ind = 1
        for i in range(0,len(props)):
            if area<props[i].filled_area:
                area = props[i].filled_area
                ind = i+1
        breast_mask = np.zeros(np.shape(image))
        breast_mask[labelim==ind]=1  
        labelim = label(breast_mask)       
        props =  regionprops(labelim)
        boundingbox = props[0].bbox
#        crop the breast mask and mammogram
        if if_crop == 1:
            breast_mask = breast_mask[boundingbox[0]:boundingbox[2],boundingbox[1]:boundingbox[3]]
            breast_raw_image = image[boundingbox[0]:boundingbox[2],boundingbox[1]:boundingbox[3]]
            lesion_mask = lesion_mask[boundingbox[0]:boundingbox[2],boundingbox[1]:boundingbox[3]]
        else:
            breast_raw_image = image
#        breast_image = rescale2uint8(breast_raw_image,breast_mask)
        breast_image = rescale2uint16(breast_raw_image,breast_mask)
        return Preprocess(breast_raw_image,breast_image,breast_mask,lesion_mask)

        
#To normalize the INbreast mammogram to 16bit   
def rescale2uint8(image,breast_mask):
    intensity_in_mask = image[breast_mask>0]
#    use top 0.2 percentile to do the strech
    maxi = np.percentile(intensity_in_mask,99.8)#np.max(intensity_in_mask)
    mini = np.percentile(intensity_in_mask,0.2)#np.min(intensity_in_mask)
#        stretch the image into 0~255
    
    image = 255*(image-mini)/(maxi-mini)
    image[breast_mask==0] = 0
    image[image<0] = 0
    image[image>255] = 255
    image = np.uint8(image)
          
    return image

def rescale2uint16(image,breast_mask):
    intensity_in_mask = image[breast_mask>0]
#    use top 0.2 percentile to do the strech
    maxi = np.percentile(intensity_in_mask,99.8)#np.max(intensity_in_mask)
    mini = np.percentile(intensity_in_mask,0.2)#np.min(intensity_in_mask)
#        stretch the image into 0~255
    
    image = 65535*(image-mini)/(maxi-mini)
    image[breast_mask==0] = 0
    image[image<0] = 0
    image[image>65535] = 65535
    image = np.uint16(image)
          
    return image


In [ ]:
#To pad the mammograms and masks to a square
import numpy as np
# To pad image into a square shape
def padimages(image,file_name, ratio):
    [length, width] = np.shape(image)
    if length/width>ratio:#1024/800
        print('This image needs padding.')
        add_wid = round(length*(1/ratio)-width)
        pad = np.zeros((length,add_wid))
        pad = pad.astype(image.dtype)
        if '_R_' in file_name:
        #                pad on the left
            pad_image = np.concatenate((pad,image),axis=1)
        else:
            pad_image = np.concatenate((image,pad),axis=1)
            
    return pad_image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, color, io, img_as_float

def mask_overlay(img, mask):
    alpha = 0.5

    img = img_as_float(img)
    rows, cols = img.shape
    # Construct RGB version of grey-level image
    img_color = np.dstack((img, img, img))
    img_hsv = color.rgb2hsv(img_color)
    color_mask = np.zeros((rows, cols, 3))
    color_mask1 = color_mask[:,:,1]
    color_mask1[np.where(mask>0)] = 1
    color_mask[:,:,1] = color_mask1
    color_mask_hsv = color.rgb2hsv(color_mask)
    
    # Replace the hue and saturation of the original image
    # with that of the color mask
    img_hsv[..., 0] = color_mask_hsv[..., 0]
    img_hsv[..., 1] = color_mask_hsv[..., 1] * alpha
    
    img_masked = color.hsv2rgb(img_hsv)
    
    return img_masked

In [ ]:
cd '/content/drive/MyDrive/Mammographic/Mask_r_cnn'

/content/drive/MyDrive/Mammographic/Mask_r_cnn


In [ ]:
### main Code  
import os
import numpy as np
#from Preprocess_mammo import Preprocess 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#from utility import padimages
#from skimage.segmentation import mark_boundaries
from skimage import io
from skimage.measure import label
#from overlay import mask_overlay
import timeit

start = timeit.default_timer()

import warnings
warnings.filterwarnings("ignore")

#from skimage import data, color, io, img_as_float

image_path = "scans/raw_mammogram/"
annotation_path = 'scans/raw_annotation/'

save_image_path = "scans/preprocessed_image/"  
    
save_mask_path = "scans/preprocessed_mask/"  

file_names = os.listdir(image_path)    
file_names = sorted(file_names)


for i in range(0,len(file_names)):

    print(file_names[i])
    mammo = io.imread(image_path+file_names[i],0)
   
    lesion_mask = io.imread(annotation_path+file_names[i])
    
    if np.max(lesion_mask)>=0:
#    Extract the breast profile and crop the mammogram, breast mask and the lesion mask
#    Normalize the image into 16-bit
        breast_preprocess = Preprocess.extract_breast_profile(mammo,lesion_mask,1)
               
        mammo = breast_preprocess.image
        breast_mask = breast_preprocess.mask
        lesion_mask =breast_preprocess.lesion_mask    
        
        print ('Number of lesions: '+str(np.max(np.unique(label(lesion_mask)))))
        
#   pad the image, to ensure the aspect ratio is 1:1
        pad_mammo = padimages(mammo,file_names[i],1)
        
#    save the preprocessed image

        io.imsave(save_image_path + file_names[i],pad_mammo)
        
        #if the image has more than 1 lesion, then seperate them into different masks and number them.
        
        labelim = label(lesion_mask)
        if np.max(labelim)>0:
#            if there is at least 1 lesion.
            for l in range(1,np.max(labelim+1)):
                l_mask = np.zeros(np.shape(labelim))
                l_mask = l_mask.astype(lesion_mask.dtype)
                l_mask [labelim==l] = 255
                num_nonzero = np.where(l_mask>0)
                num_nonzero = len(num_nonzero[0])

                if num_nonzero>15:
                    print('A valid mask')
#                   Pad the mask in the same way as padding the image
                    pad_l_mask = padimages(l_mask,file_names[i],1)
                    io.imsave(save_mask_path+file_names[i][:-4]+str(l)+'.png',pad_l_mask)
                else:
                    print('Has a tiny piece of noise that is not valid for training!')
                    
        else:# if there is no lesion
            pad_lesion_mask = padimages(lesion_mask,file_names[i],1)
            io.imsave(save_mask_path+file_names[i][:-4]+str(0)+'.png',pad_lesion_mask)
       
stop = timeit.default_timer()
print('RunTime per image: ', (stop - start)/ len(file_names)) 

22580192_5530d5782fc89dd7_MG_R_CC_ANON.png
Number of lesions: 1
This image needs padding.
A valid mask
This image needs padding.
RunTime per image:  4.5182228539997595
